In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from IPython.display import display

# Your watchlist & date range
tickers    = ["GME", "FTEK", "MP", "LFMD", "AZ", "FUBO", "HOVR", "AEVA", "LGCY", "GSAT", "ATRO"]
end_date   = datetime.today()
start_date = end_date - timedelta(days=180)

def calculate_indicators(df):
    # Ensure 'Close' column exists and is numeric
    if 'Close' not in df.columns or not pd.api.types.is_numeric_dtype(df['Close']):
        raise ValueError("DataFrame missing 'Close' column or contains non-numeric data")
    
    # Convert 'Close' to numeric, coercing errors to NaN
    df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
    
    # Drop any rows with NaN in 'Close' early to avoid issues
    df = df.dropna(subset=['Close'])
    
    # EMAs
    df['EMA10'] = df['Close'].ewm(span=10, adjust=False).mean()
    df['EMA20'] = df['Close'].ewm(span=20, adjust=False).mean()
    df['EMA50'] = df['Close'].ewm(span=50, adjust=False).mean()
    
    # MACD histogram
    exp1 = df['Close'].ewm(span=12, adjust=False).mean()
    exp2 = df['Close'].ewm(span=26, adjust=False).mean()
    
    # Ensure exp1 and exp2 have the same index
    if not exp1.index.equals(exp2.index):
        raise ValueError("Index misalignment between exp1 and exp2")
    
    macd = exp1 - exp2
    signal = macd.ewm(span=9, adjust=False).mean()
    df['MACD_Hist'] = macd - signal
    
    # 20-day avg volume
    df['Vol_Avg20'] = df['Volume'].rolling(window=20).mean()
    
    return df.dropna()

results = []

for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date, progress=False)
    if data.empty or len(data) < 60:
        continue

    try:
        df = calculate_indicators(data)

        # Scalar value extraction via iloc
        close     = df['Close'].iloc[-1]
        ema10     = df['EMA10'].iloc[-1]
        ema20     = df['EMA20'].iloc[-1]
        ema50     = df['EMA50'].iloc[-1]
        macd_hist = df['MACD_Hist'].iloc[-1]
        volume    = df['Volume'].iloc[-1]
        vol_avg20 = df['Vol_Avg20'].iloc[-1]

        # Compute 60-day resistance (excluding latest bar)
        resistance = df['Close'].iloc[-61:-1].max()

        # Boolean flags
        trend_ok    = (close > ema10) and (ema10 > ema20) and (ema20 > ema50)
        mom_ok      = macd_hist > 0
        vol_ok      = volume < vol_avg20
        breakout_ok = close > resistance

        if trend_ok and mom_ok and vol_ok and breakout_ok:
            support     = max(ema20, ema50)
            support_dist = (close - support) / close
            results.append({
                "Ticker": ticker,
                "Close": round(close, 2),
                "Resistance": round(resistance, 2),
                "Support": round(support, 2),
                "Support_Dist": round(support_dist, 3),
                "MACD_Hist": round(macd_hist, 3),
                "Volume": int(volume),
                "Vol_Avg20": int(vol_avg20)
            })

    except Exception as e:
        print(f"Error processing ticker {ticker}: {str(e)}")
        continue

# Compile & sort
df_results = pd.DataFrame(results)
if not df_results.empty:
    df_results = df_results.sort_values("Support_Dist", ascending=False).reset_index(drop=True)

# Display
display(df_results)

/var/folders/4x/6rrj55450t34hsq3pbqbh0300000gn/T/ipykernel_49997/1343331935.py:31: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date, end=end_date, progress=False)


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().